In [1]:
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import pyodbc
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
# Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)


In [3]:
# Open Sheet
sheet = client.open_by_key("1vadc79ujOS66vVhIhRlieVafQKMBbaixvvSNFU0mJRQ") # Master Sheets ( Dimensions Sheet)

#### Extracting Masters from GoogleSheetBackend Sheets - Managed by GCPL MIS Exec

In [4]:
StoreMaster = sheet.worksheet("StoreMaster")
SOSTargetMaster = sheet.worksheet("SOSTargetMaster")
MSLMaster = sheet.worksheet("MSLMaster")

In [5]:
StoreMaster = get_as_dataframe(StoreMaster)
SOSTargetMaster = get_as_dataframe(SOSTargetMaster)
MSLMaster = get_as_dataframe(MSLMaster)

#### Connection with SQL Server ( Optional for Now) once SQL database corrected , will be connected to Server 

In [6]:
server = '157.119.230.120,4071'    
database = 'PowerBI_LISV2'      
username = 'sa'
password = 'Y0m@Sql.!.123'

# Connection string
conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""

## Data Collection

### CategoryTracking

In [7]:
# conn = pyodbc.connect(conn_str)
query = """ SELECT [CategoryTrackingAIId]
      ,[VisitID]
      ,[SubmittedDate&Time] AS Date
      ,[SyncDate&Time]
      ,[Month]
      ,[Year]
      ,[Region]
      ,[State]
      ,[City]
      ,[RSMCode]
      ,[BranchCode]
      ,[ASMCode]
      ,[SOCode]
      ,[SupervisorCode]
      ,[ISPCode]
      ,[UserName]
      ,[FullName]
      ,[Designation]
      ,[ClientStoreCode]
      ,[LISStoreCode]
      ,[StoreName]
      ,[StoreAddress]
      ,'SAMT' as [Format]
      ,[ChainCode]
      ,[ChainName]
      ,[SubChainName]
      ,'GT' as [StoreType]
      ,[StoreClub]
      ,[DistributorCode]
      ,[DistributorName]
      ,[PJPStatus]
      ,[CategoryTrackingCode]
      ,[CategoryTrackingName]
      ,[CategoryTrackingCode] as [CategoryProductCode]
      ,[CategoryTrackingName] as [CategoryProductName]
      ,[IscategoryAvailable]
      ,[ReasonforNonavailability] AS 'Non Availability Reason'
      ,[Imagecapturingenabled]
      ,[AppQuestions]
      ,[AppResponse]
      ,[AuditQuestions]
      ,[AuditResponse]
      ,[AuditResponsePoints]
      ,[AIDetectionStatus]
      ,[AIdetectionreason]
      ,[StackCount]
      ,[DepthCount]
      ,[ShelfCount]
      ,[Facings]
      ,[TotalFacings]
      ,[FacingContribution]
      ,[TgtCatSOS]
      ,[SOSAch]
      ,[Score]
      ,[CountofUniqueSelfProductFacingsDetected] AS '#UniqueSelfProductFacing'
      ,[CountofUniqueCompetitionProductFacingsdetected] AS '#UniqueComprtitionProductFacing'
      ,[SelfProductFacingsDetected]
      ,[CompetitionProductsDetected]
      ,[TotalFacingsDetected]
      ,[FacingContribution(AI)]
      ,[FinalImagePath]
      ,[AnnotatedImagePath]
      ,[Latitude]
      ,[Longitude]
  FROM [PowerBI_LISV2].[GCPL_SAMTManagement].[CategoryTrackingAIReportExport]
  where Month([SubmittedDate&Time])= MONTH(GETDATE()) 
  AND ([AIDetectionStatus] in ('Yes','No') OR ([AIDetectionStatus] IS NULL))
  AND [CategoryTrackingCode] IN ('AERAEROSOL-01','Aerosol-01','AERPocket-01','HAIRCOLOR-01','HITAEROSOL-01','LIQMACHREF-01','LIQUIDDETERGENT-01','Liquids-01','SOAPS-01')
  ;

# """
  

In [8]:
# CategoryTrackingAIReportExport = pd.read_sql(query, conn)
# conn.close()

#### CSV CategoryTrackingAI

In [9]:
# CSV Read Module
CTReportExportAI = pd.read_excel(r"C:\Users\Administrator\LIS\GCPL SAMT AI\CTReportExportAI_23-05-2025_1_22.xls")
# CTReportExportAI= pd.read_excel(r'C:\Users\Administrator\LIS\GCPL SAMT AI\CTReportExportAI_21-05-2025.xls')
CTReportExportAI.rename(columns= {'Submitted Date&Time':'Date','Sync Date&Time':'SyncDate&Time','LISStoreCode ':'LISStoreCode','Category Tracking Code':'CategoryTrackingCode','Chain Name':'ChainName',
                                  'Reason for Non availability' :'Non Availability Reason','AI detection reason' : 'AIdetectionreason',
                                 'AI Detection Status': 'AIDetectionStatus'}, inplace=True)

# SQL Based filters
CTReportExportAI_1 = CTReportExportAI[
    (
        CTReportExportAI['AIDetectionStatus'].isin(['Yes', 'No']) | 
        CTReportExportAI['AIDetectionStatus'].isna()
    ) &
    CTReportExportAI['CategoryTrackingCode'].isin([
        'AERAEROSOL-01', 'Aerosol-01', 'AERPocket-01',
        'HAIRCOLOR-01', 'HITAEROSOL-01', 'LIQMACHREF-01',
        'LIQUIDDETERGENT-01', 'Liquids-01', 'SOAPS-01'
    ])
]

CategoryTrackingAIReportExport=CTReportExportAI_1


In [10]:
CategoryTrackingAIReportExport=CategoryTrackingAIReportExport.drop(columns=['SyncDate&Time', 'Month', 'Year'])

In [11]:
# CategoryTrackingAIReportExport['AuditQuestions'].unique

In [12]:
CategoryTrackingAIReportExport.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25168 entries, 0 to 39232
Data columns (total 60 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Visit ID                                              25168 non-null  int64  
 1   Date                                                  25168 non-null  object 
 2   Region                                                25168 non-null  object 
 3   State                                                 25168 non-null  object 
 4   City                                                  25168 non-null  object 
 5   RSMCode                                               0 non-null      float64
 6   BranchCode                                            0 non-null      float64
 7   ASMCode                                               22106 non-null  object 
 8   SOCode                                                0 non-n

In [13]:
GCPL_StoreMaster1 = pd.read_csv(r'C:\Users\Administrator\LIS\GCPL SAMT AI\GCPLStoreMaster.csv', encoding='cp1252')

In [14]:
# GCPL_StoreMaster = pd.read_csv(r'C:\Users\Administrator\LIS\GCPL SAMT AI\GCPLStoreMaster.csv', encoding='cp1252', 
#                                usecols = ['LIS Store Code','Supervisor Name','Store Status','RKAM','ASM','SO Name'])

In [15]:
GCPL_StoreMaster= StoreMaster[['LIS Store Code','Supervisor Name','Store Status','RKAM','ASM','SO Name']]

In [16]:
GCPL_StoreMaster.head(2)

LIS Store Code Supervisor Name Store Status  RKAM         ASM      SO Name
0   GCPL-SAMT-07       David Raj       Active   NaN  Raja Durai  Karthikeyan
1  GCPL-SAMT-999       David Raj       Active   NaN  Raja Durai  Karthikeyan

In [17]:
CategoryTrackingAIReportExport2 = pd.merge(CategoryTrackingAIReportExport,GCPL_StoreMaster,how='left',left_on='LIS Store Code',
                                           right_on='LIS Store Code')

# LIS Store Code NULL wala condition to be added

In [18]:
# CategoryTrackingAIReportExport2.info()

In [19]:
# MasterSOSTarget = pd.read_excel(r'C:\Users\Administrator\LIS\GCPL SAMT AI\Master_SOSTarget.xlsx', usecols = ['Category Tracking Code','Start Date'])

In [20]:
MasterSOSTarget = SOSTargetMaster[['Category Tracking Code','Start Date']]

In [21]:
MasterSOSTarget.head()

Category Tracking Code Start Date
0          AERAEROSOL-01  01-Sep-24
1              AERCar-01  01-Sep-24
2            AERMatic-01  01-Sep-24
3             Aerosol-01  01-Sep-24
4           AERPocket-01  01-Sep-24

In [22]:
CategoryTrackingAIReportExport3 = pd.merge(CategoryTrackingAIReportExport2,MasterSOSTarget,how='left',left_on='CategoryTrackingCode',
                                           right_on='Category Tracking Code')

In [23]:
CategoryTrackingAIReportExport3['Date'] = pd.to_datetime(CategoryTrackingAIReportExport3['Date'])
CategoryTrackingAIReportExport3['Start Date'] = pd.to_datetime(CategoryTrackingAIReportExport3['Start Date'])
CategoryTrackingAIReportExport3['CatStartCheck'] = CategoryTrackingAIReportExport3.apply(lambda row: 1 if row['Date'] >= row['Start Date'] else 0,axis=1)

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_6252\1784011439.py:1: UserWarning: Parsing dates in %d-%m-%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  CategoryTrackingAIReportExport3['Date'] = pd.to_datetime(CategoryTrackingAIReportExport3['Date'])


In [24]:
CategoryTrackingAIReportExport3_filtered = CategoryTrackingAIReportExport3[
    (CategoryTrackingAIReportExport3['CatStartCheck'] == 1) &
    (~CategoryTrackingAIReportExport3['ChainName'].isin(['Cp Wholesale', 'D Mart', 'Metro C&C', 'Walmart C&C']))
]
# can also be done in SQL 

In [25]:
CategoryTrackingAIReportExport3_filtered=CategoryTrackingAIReportExport3_filtered.fillna(0)

In [26]:
df = CategoryTrackingAIReportExport3_filtered
df_active = df[df['Store Status'] == 'Active']


In [27]:
df.rename(columns={"Non Availability Reason": "Non Availability Reason-old"}, inplace=True)

In [28]:
# df.info()

In [29]:
df['Visit ID'].count()

25168

In [30]:
df['Non Availability Reason'] = df['Non Availability Reason-old'].combine_first(df['AIdetectionreason'])

In [31]:
df['Visit ID'].count()

25168

In [32]:
df['Key'] = df['Date'].dt.strftime('%d/%m/%Y') + df['LIS Store Code'] + df['Category Tracking Code'].astype(str)
# df = df.drop(columns=["Non Availability Reason-old", "Start Date", "CatStartCheck"])

In [33]:
df.to_csv(r'C:\Users\Administrator\LIS\GCPL SAMT AI\test.csv', index=False)

In [34]:
df.head()

Visit ID                Date Region       State     City  RSMCode  \
0     79725 2025-05-22 14:37:37  South  Tamil Nadu  Chennai      0.0   
1     79725 2025-05-22 14:37:37  South  Tamil Nadu  Chennai      0.0   
2     79725 2025-05-22 14:37:37  South  Tamil Nadu  Chennai      0.0   
3     79725 2025-05-22 14:37:37  South  Tamil Nadu  Chennai      0.0   
4     79725 2025-05-22 14:37:37  South  Tamil Nadu  Chennai      0.0   

   BranchCode         ASMCode  SOCode SupervisorCode  ... Supervisor Name  \
0         0.0  GCPLSAMT-FO-01     0.0    GCPL-SUP-02  ...       David Raj   
1         0.0  GCPLSAMT-FO-01     0.0    GCPL-SUP-02  ...       David Raj   
2         0.0  GCPLSAMT-FO-01     0.0    GCPL-SUP-02  ...       David Raj   
3         0.0  GCPLSAMT-FO-01     0.0    GCPL-SUP-02  ...       David Raj   
4         0.0  GCPLSAMT-FO-01     0.0    GCPL-SUP-02  ...       David Raj   

  Store Status RKAM         ASM      SO Name Category Tracking Code  \
0       Active  0.0  Raja Durai  Karthikeyan          AERAEROSOL-01   
1       Active  0.0  Raja Durai  Karthikeyan             Aerosol-01   
2       Active  0.0  Raja Durai  Karthikeyan           AERPocket-01   
3       Active  0.0  Raja Durai  Karthikeyan           HAIRCOLOR-01   
4       Active  0.0  Raja Durai  Karthikeyan          HITAEROSOL-01   

  Start Date CatStartCheck Non Availability Reason  \
0 2024-09-01             1                       0   
1 2024-09-01             1                       0   
2 2024-09-01             1                       0   
3 2024-09-01             1                       0   
4 2024-09-01             1                       0   

                                    Key  
0  22/05/2025GCPL-SAMT-242AERAEROSOL-01  
1     22/05/2025GCPL-SAMT-242Aerosol-01  
2   22/05/2025GCPL-SAMT-242AERPocket-01  
3   22/05/2025GCPL-SAMT-242HAIRCOLOR-01  
4  22/05/2025GCPL-SAMT-242HITAEROSOL-01  

[5 rows x 70 columns]

In [35]:
df = df.drop_duplicates(subset='Key').reset_index(drop=True)

In [36]:
df['Visit ID'].count()

25163

In [37]:

df['Format2'] = 'SAMT'
df['ImagePath'] =''
df['Category Type']='Format'
df['SOS TGT']=''
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y %H:%M:%S')
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['ChainName'] = df['ChainName'].str.replace(r'^SAMT\s+', '', regex=True)
df['SOSAch%_']= df['Facing Contribution']


In [38]:
df['AISOSAch%_'] = (
    (df['Self Product Facings Detected'] / df['Total Facings']).replace([np.inf, -np.inf], 0).fillna(0) * 100
).round(0).clip(upper=100)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25163 entries, 0 to 25162
Data columns (total 78 columns):
 #   Column                                                Non-Null Count  Dtype         
---  ------                                                --------------  -----         
 0   Visit ID                                              25163 non-null  int64         
 1   Date                                                  25163 non-null  datetime64[ns]
 2   Region                                                25163 non-null  object        
 3   State                                                 25163 non-null  object        
 4   City                                                  25163 non-null  object        
 5   RSMCode                                               25163 non-null  float64       
 6   BranchCode                                            25163 non-null  float64       
 7   ASMCode                                               25163 non-null  object

In [40]:
### Columns ReOrdering
df=df[['Visit ID','Date','Region','State','RKAM','ASMCode','SO Name','SupervisorCode','Supervisor Name','UserName','FullName','Month','Year','Format2','Chain Code','ChainName','LIS Store Code','Store Name','Is category Available','Non Availability Reason','Audit Response','Category Tracking Name','Final Image Path','Annotated Image Path','ImagePath','Category Type','SOS TGT','Facings','Total Facings','Facing Contribution','SOSAch%_','Self Product Facings Detected','Competition Products Detected','Total Facings Detected','AISOSAch%_']]


In [35]:
# df['SOSAch%_']= df['Facing Contribution']

In [41]:
df['Visit ID'].count()

25163

In [42]:
df.to_csv(r'C:\Users\Administrator\LIS\GCPL SAMT AI\AICAtegoryTrackingSOS_ModelOutput.csv', index=False)

#### End of SOS

In [43]:
df_VisitID = pd.DataFrame(df['Visit ID'].unique(), columns=['Visit ID'])

In [44]:
df_VisitID.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2943 entries, 0 to 2942
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Visit ID  2943 non-null   int64
dtypes: int64(1)
memory usage: 23.1 KB


## Primary ShelfStock

In [45]:
PrimaryShelfStock = pd.read_excel(r"C:\Users\Administrator\LIS\GCPL SAMT AI\CTSelfProductAvailabilityExportAI_23-05-2025.xls")

In [46]:
PrimaryShelfStock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159628 entries, 0 to 159627
Data columns (total 47 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Visit ID                     159628 non-null  int64  
 1   Submitted Date&Time          159628 non-null  object 
 2   Sync Date&Time               159628 non-null  object 
 3   Month                        159628 non-null  int64  
 4   Year                         159628 non-null  int64  
 5   Region                       159628 non-null  object 
 6   State                        159628 non-null  object 
 7   City                         159628 non-null  object 
 8   RSMCode                      0 non-null       float64
 9   BranchCode                   0 non-null       float64
 10  ASMCode                      140514 non-null  object 
 11  SOCode                       0 non-null       float64
 12  SupervisorCode               159039 non-null  object 
 13 

In [47]:
PrimaryShelfStock['is_check'] = PrimaryShelfStock['Visit ID'].isin(df_VisitID['Visit ID']).astype(int)

In [48]:
PrimaryShelfStock = PrimaryShelfStock[PrimaryShelfStock['is_check']==1]

In [49]:
print(PrimaryShelfStock['Visit ID'].count())

159460


In [50]:
PrimaryShelfStock2 = PrimaryShelfStock[(PrimaryShelfStock['Visit ID'].notna()) & (PrimaryShelfStock['Product Type'] == 'Self')]

In [51]:
PrimaryShelfStock2 = PrimaryShelfStock2.drop(columns=["Sync Date&Time", "Month", "Year"], errors='ignore')
PrimaryShelfStock2 = PrimaryShelfStock2.rename(columns={"Submitted Date&Time": "Date"})

In [52]:
PrimaryShelfStock2['Facings Detected Check'] = PrimaryShelfStock2['Facings Detected'].apply(lambda x: 1 if x > 0 else 0)


In [53]:
print(PrimaryShelfStock2['Visit ID'].count())

155103


In [54]:
MSLMaster['key'] = MSLMaster["Chain Code"]+MSLMaster["Category Tracking Code"]+MSLMaster["Roboflow Name"]

In [55]:
MSLMaster.info()

<class 'pandas.core.frame.DataFrame'>
Index: 304 entries, 0 to 303
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Category Tracking Code  304 non-null    object
 1   Category Tracking Name  304 non-null    object
 2   Category Product Code   304 non-null    object
 3   Category Product Name   304 non-null    object
 4   Product Category        304 non-null    object
 5   Product Brand           304 non-null    object
 6   Product ID              304 non-null    object
 7   Product Desc            304 non-null    object
 8   SKU                     304 non-null    object
 9   Roboflow Name           304 non-null    object
 10  Chain Code              304 non-null    object
 11  Chain Name              304 non-null    object
 12  MSL                     304 non-null    object
 13  key                     304 non-null    object
dtypes: object(14)
memory usage: 35.6+ KB


In [57]:
MSLMaster_renamed = MSLMaster.rename(columns={"Roboflow Name": "Product Name"})

PrimaryShelfStock3 = pd.merge(PrimaryShelfStock2,
    MSLMaster_renamed,
    on=["Chain Code", "Category Tracking Code", "Product Name"],
    how="left"
)

In [58]:
print(PrimaryShelfStock3['Visit ID'].count())

156183


In [59]:
MSLMaster_renamed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 304 entries, 0 to 303
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Category Tracking Code  304 non-null    object
 1   Category Tracking Name  304 non-null    object
 2   Category Product Code   304 non-null    object
 3   Category Product Name   304 non-null    object
 4   Product Category        304 non-null    object
 5   Product Brand           304 non-null    object
 6   Product ID              304 non-null    object
 7   Product Desc            304 non-null    object
 8   SKU                     304 non-null    object
 9   Product Name            304 non-null    object
 10  Chain Code              304 non-null    object
 11  Chain Name              304 non-null    object
 12  MSL                     304 non-null    object
 13  key                     304 non-null    object
dtypes: object(14)
memory usage: 35.6+ KB


In [60]:
PrimaryShelfStock3.to_csv(r'C:\Users\Administrator\LIS\GCPL SAMT AI\PrimaryShelfStock3.csv', index=False)

In [61]:
PrimaryShelfStock4 = PrimaryShelfStock3[PrimaryShelfStock3['MSL'] == 'Yes'] # MSL Fltered Data

In [63]:
print(PrimaryShelfStock3['Visit ID'].count())

156183


#### SKU Master

In [64]:
SKUMaster = pd.merge(
    StoreMaster, MSLMaster,left_on="Distributor Code",right_on="Chain Code",how="left",suffixes=('', '_Master_MSLTarget')
)

In [65]:
MSL = PrimaryShelfStock4[['Visit ID','Date','LIS Store Code']]

In [66]:
MSL=MSL.drop_duplicates()

In [67]:
MSL.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2927 entries, 3 to 156129
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Visit ID        2927 non-null   int64 
 1   Date            2927 non-null   object
 2   LIS Store Code  2927 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.5+ KB


In [68]:
MSL_SKUMaster = pd.merge(MSL,SKUMaster,on="LIS Store Code",how="left",suffixes=('', '_SKU_Master'))

In [69]:
# MSL_SKUMaster.info() # 124910

In [70]:
MSL_SKUMaster2 = pd.merge(
    MSL_SKUMaster,
    PrimaryShelfStock3,
    left_on=["Date", "LIS Store Code", "Roboflow Name"],
    right_on=["Date", "LIS Store Code", "Product Name"],
    how="left",
    suffixes=('', '_PrimaryShelf')
)

In [71]:
MSL_SKUMaster3 = pd.merge(
    MSL_SKUMaster2,
    SOSTargetMaster,
    left_on="Category Tracking Code",
    right_on="Category Tracking Code",
    how="left",
    suffixes=('', '_Master_SOSTarget')
)

In [72]:
MSL_SKUMaster2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150836 entries, 0 to 150835
Data columns (total 97 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   Visit ID                             150836 non-null  int64  
 1   Date                                 150836 non-null  object 
 2   LIS Store Code                       150836 non-null  object 
 3   Region                               150834 non-null  object 
 4   State                                150834 non-null  object 
 5   City                                 150834 non-null  object 
 6   Distributor Code                     150834 non-null  object 
 7   Distributor                          103472 non-null  object 
 8   Chain                                150834 non-null  object 
 9   Format                               150834 non-null  object 
 10  Store Type                           150834 non-null  object 
 11  Client Store 

In [73]:
MSLFinal = MSL_SKUMaster3[['Visit ID','Date','LIS Store Code','Region','State','City','Distributor Code',
            'Client Store Code','Store Name','Supervisor Name','RKAM','ASM','Supervisor Code','SO Name'
                          ,'Category Product Name_x','Category Tracking Code','Category Tracking Name_x',
                          'MSL','Roboflow Name','Final Image Path','Facings Detected Check','Chain','Start Date']]

In [74]:
MSLFinal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150836 entries, 0 to 150835
Data columns (total 23 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Visit ID                  150836 non-null  int64  
 1   Date                      150836 non-null  object 
 2   LIS Store Code            150836 non-null  object 
 3   Region                    150834 non-null  object 
 4   State                     150834 non-null  object 
 5   City                      150834 non-null  object 
 6   Distributor Code          150834 non-null  object 
 7   Client Store Code         150834 non-null  object 
 8   Store Name                150834 non-null  object 
 9   Supervisor Name           150834 non-null  object 
 10  RKAM                      0 non-null       float64
 11  ASM                       150834 non-null  object 
 12  Supervisor Code           0 non-null       float64
 13  SO Name                   150834 non-null  o

In [76]:
MSLFinal.to_csv(r'C:\Users\Administrator\LIS\GCPL SAMT AI\MSLFinal.csv', index=False)